In [37]:
%serialconnect



***Connecton broken [Input/output error]
You may need to reconnectserial exception on close write failed: [Errno 5] Input/output error

Closing serial Serial<id=0x7f3996e396d8, open=True>(port='/dev/ttyUSB2', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)
Connecting to Serial /dev/ttyUSB0 baud=115200 
Ready.

In [38]:
# An ADC (Analog to Digital Converter) is needed to read the value from each green light pulse sensor
# Not every pin on the ESP32 has this capability.

# Here we test which Pins can do ADC
import machine

for i in range(50):
    try:
        machine.ADC(machine.Pin(i))
        print("ADC works on Pin", i)
    except ValueError:
        pass
    

ADC works on Pin 32
ADC works on Pin 33
ADC works on Pin 34
ADC works on Pin 35
ADC works on Pin 36
ADC works on Pin 37
ADC works on Pin 38
ADC works on Pin 39


In [13]:
# With the USB socket pointing away from you, the only pins that will 
# be good for this are on the right hand side, and are D33, D32, D35, D34

In [39]:
# we need to set the working range to between 0 and 5V using the atten() function
a34 = machine.ADC(machine.Pin(34))
a34.atten(machine.ADC.ATTN_11DB)

In [40]:
# We can make a short set of readings, like so:
import time

for i in range(10):
    print(a34.read())
    time.sleep(0.1)

1811
1837
1792
1715
1865
1852
1828
1835
1834
1834


In [16]:
# We can efficiently print out the readings (without overloading the browser 
# by printing it in an unlimited stream), like so:

import time

for i in range(60):
    for j in range(100):
        print(a34.read(), end=" ")  # puts a blank instead of a newline
        time.sleep(0.01)  # make a reading every 1/100th of a second
    print()

# You can stop the printing by selecting Kernel -> Interrupt in the menu, 
# or hitting escape and then "i" twice.

2131 2193 2229 2234 2211 2166 2112 2037 1965 1887 1820 1755 1692 1642 1605 1585 1581 1584 1600 1616 1643 1649 1662 1663 1662 1664 1671 1680 1698 1721 1753 1781 1819 1850 1875 1904 1931 1931 1919 1889 1853 1806 1751 1706 1655 1617 1595 1584 1584 1590 1613 1639 1680 1728 1785 1840 1886 1931 1975 2009 2046 2086 2122 2145 2167 2181 2193 2203 2210 2222 2215 2224 2225 2225 2231 2235 2240 2255 2285 2331 2387 2466 2545 2639 2699 2734 2736 2722 2687 2640 2587 2522 2455 2390 2362 2405 2578 3043 4091 4095 
4095 4095 4095 4095 4095 2982 2125 1322 672 163 0 0 0 0 128 659 1238 1807 2319 2751 3071 3227 3125 2787 2297 1744 1179 624 80 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 130 594 1040 1424 1758 2019 2224 2352 2433 2458 2451 2413 2355 2275 2203 2119 2038 1965 1911 1843 1815 1779 1767 1744 1754 1743 1758 1757 1775 1774 1788 1792 1809 1809 1821 1819 1833 1827 1841 1837 1845 1842 1845 1841 1854 1841 1847 1846 1851 1841 
1849 1848 1846 1854 1848 1852 1854 1855 1853 1856 1851 1856 1855 1856 1860 1860 18

Traceback (most recent call last):
  File "<stdin>", line 9, in <module>
KeyboardInterrupt: 


In [9]:
print(time.ticks_ms())


2908109


In [17]:
%capture --quiet testpulsesingle.txt

# Make 15 seconds of samples at 100 samples per second
import time

for i in range(15):
    for j in range(100):
        print(a34.read(), end=" ")  # puts a blank instead of a newline
        time.sleep(0.01)
    print()
    

15 lines captured

In [ ]:
# To plot what these readings are, you need to open up the 
# notebook "plot_adc_measurements" which runs in Python on your PC
# (instead of the Python in the ESP32) and allows you to do scientific plots


In [10]:
import time

def measure34(numberofseconds, samplespersecond):
    ticks0 = time.ticks_ms()
    for i in range(numberofseconds):
        for j in range(samplespersecond):
            print(a34.read(), end=" ")  # puts a blank instead of a newline
            time.sleep(1/samplespersecond)
        print()
    return (time.ticks_ms() - ticks0)/1000
    



***Connecton broken [Input/output error]
You may need to reconnect

***OSError [Input/output error]



In [8]:
%capture --quiet testpulsesingle100.txt
numberofseconds = 100
elapsedtime = measure34(10, numberofseconds)

10 lines captured

In [9]:
print(elapsedtime, "should be close to", numberofseconds)

10.213 should be close to 100


In [20]:
%capture --quiet testpulsesingle500.txt
elapsedtime = measure34(10, 500)

10 lines captured

In [21]:
print(elapsedtime, "should be close to", numberofseconds)

11.034 should be close to 10


In [ ]:
# There seems to be high frequency noise in the sensor
# So this lets us make 10 readings at a time and then 
# print them out at a frequency of 50 samples per second
# which gives a pretty smooth result

In [28]:
import time

def measure34sub(numberofseconds, samplespersecond, subsamplestoaverage):
    ticks0 = time.ticks_ms()
    microsecondstep = int(1/samplespersecond/subsamplestoaverage*1000000)
    for i in range(numberofseconds):
        for j in range(samplespersecond):
            samplesum = 0
            for k in range(subsamplestoaverage):
                samplesum += a34.read()
                time.sleep_us(microsecondstep)
            print(int(samplesum/subsamplestoaverage+0.5), end=" ")
        print()
    return (time.ticks_ms() - ticks0)/1000
    

In [29]:
%capture --quiet testpulsesingle50avg.txt

numberofseconds = 10
elapsedtime = measure34sub(numberofseconds, 50, 10)


10 lines captured

In [30]:
print(elapsedtime, "should be close to", numberofseconds)

10.262 should be close to 10


In [41]:

########################
# Here we Try lighting an LED on pin 5
# by an intensity using pulse width modulation
m5 = machine.PWM(machine.Pin(5))


In [42]:
# brightest
m5.duty(1023)  

# middle
m5.duty(500)  

# dark
m5.duty(50)  


In [43]:
# then we copy the reading into the led brightness
# scaling by a range
alow, ahigh = 1500, 1900
import time
for i in range(100000):
    x = a34.read()
    x = x - alow
    x = x/(ahigh-alow)
    if x < 0:
        x = 0
    if x > 1:
        x = 1
    x = 1 - x
    m5.duty(int(x*1023))
    time.sleep(0.05)

...
**[ys] <class 'serial.serialutil.SerialException'>
**[ys] read failed: device reports readiness to read but returned no data (device disconnected or multiple access on port?)


**[ys] <class 'serial.serialutil.SerialException'>
**[ys] read failed: device reports readiness to read but returned no data (device disconnected or multiple access on port?)

